<a href="https://colab.research.google.com/github/scottpauls/genai-teaching-tools/blob/main/RAG_eg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
import os
os.kill(os.getpid(), 9)

In [5]:
# prompt: Write minimal code using langchain for a RAG pipeline that reads data from a list of files
!pip install faiss-cpu
!pip install -U langchain-community
!pip install tiktoken
import os
import langchain

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from google.colab import userdata

# Replace with your actual file paths
file_paths = ["txt1.txt"]

# Load documents
documents = []
for file_path in file_paths:
    loader = TextLoader(file_path)
    documents.extend(loader.load())

# Split documents
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Create embeddings and vectorstore
# Get the API key from the environment variable or prompt the user
import getpass
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
if OPENAI_API_KEY is None:
    OPENAI_API_KEY = getpass.getpass("Enter your OpenAI API key: ")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY #Set the key in the environment variable.
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY) #Pass the key directly to OpenAIEmbeddings
db = FAISS.from_documents(texts, embeddings)

# Create retrieval QA chain
qa = RetrievalQA.from_chain_type(llm=OpenAI(openai_api_key=OPENAI_API_KEY), chain_type="stuff", retriever=db.as_retriever()) #Pass the key to OpenAI

# Example query
query = "What is the main topic of these files?"
result = qa.run(query)
result

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.6 MB/s eta 0:00:00


<ipython-input-5-3d8583cfd5b5>:40: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  qa = RetrievalQA.from_chain_type(llm=OpenAI(openai_api_key=OPENAI_API_KEY), chain_type="stuff", retriever=db.as_retriever()) #Pass the key to OpenAI
<ipython-input-5-3d8583cfd5b5>:44: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa.run(query)


' The main topic of these files is principles of learning and teaching.'

In [6]:
query = "How can I change my teaching to be more effective?"
result = qa.run(query)
result

" Based on the principles listed above, some ways to change your teaching to be more effective could include regularly gathering and using information about your students' prior knowledge and motivations, aligning your course design and instructional activities with your learning objectives and assessments, setting clear expectations for students, prioritizing the most important aspects of your subject matter, recognizing and addressing any potential biases or blind spots you may have as an expert, and constantly reflecting and seeking feedback to refine your teaching methods."

In [7]:
query = "How can I change can I identify my own biaes?"
result = qa.run(query)
result

' To identify your own biases, you can practice self-reflection and actively seek feedback from others. Additionally, you can also engage in diversity and inclusion training or workshops to broaden your perspective and understanding of different experiences.'